In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score as _ARI

In [12]:
# import data
filename_results = '/Users/chrismader/Python/SLDS/Output/gridsearch_results.csv'
filename_static = '/Users/chrismader/Python/SLDS/Data/static_data.csv'
res = pd.read_csv(filename_results)
static = pd.read_csv(filename_static, usecols=[0,1,2])

In [14]:
# prep data: add sectors
res['sector'] = res['security'].map(static.set_index('Ticker')['SectorID'])
res.insert(res.columns.get_loc('security') + 1, 'sector', res.pop('sector'))
print(res.columns)  #.to_list()
print(list(np.unique(res.sector)))

Index(['security', 'sector', 'config', 'rank', 'score', 'dt', 'n_regimes',
       'dim_latent', 'single_subspace', 'train_window', 'overlap_window',
       'avg_inferred_regime_length', 'elbo_start (min all runs)',
       'elbo_end (max all runs)', 'elbo_delta (max all runs)',
       'cpll (max all runs)', 'max cpll (proxy bound, paired)', 'mode_usage',
       'cagr_rel', 'cagr_strat', 'cagr_bench', 'cagr_rel_cusum',
       'cagr_strat_cusum', 'cagr_bench_cusum', 'cagr_rel_ex_ante',
       'cagr_strat_ex_ante', 'cagr_bench_ex_ante'],
      dtype='object')
['IT', 'TC']


In [16]:
# Relative CAGR

n_regimes_sel = [3, 4]        # rows
sectors_sel   = list(np.unique(res.sector))  # ['CD', 'CS', 'FN']

res_f = res.loc[
    res['n_regimes'].isin(n_regimes_sel) & res['sector'].isin(sectors_sel),
    ['config','n_regimes','dim_latent','sector','score']]

# pivot on filtered data
pt = res_f.pivot_table(
    index=['config','n_regimes','dim_latent'],
    columns='sector',
    values='score',
    aggfunc='mean').sort_index().sort_index(axis=1)

# drop rows with any inf in selected sectors, then row-mean
finite_mask = np.isfinite(pt.to_numpy()).all(axis=1)
pt_sel = pt.loc[finite_mask].copy()
pt_sel['avg'] = pt_sel.mean(axis=1, skipna=True)

# sort and style
pt_sel = pt_sel.sort_values('avg', ascending=False)
styled_sel = (
    pt_sel.style
         .format('{:.3f}')
         .background_gradient(cmap='RdYlGn', vmin=-0.1, vmax=0.1)
         .highlight_null())

styled_sel

,,sector,IT,TC,avg
config,n_regimes,dim_latent,,,
factor2_ff5,4,5,-0.017,0.040,0.011
factor2_ff3mom,4,4,0.001,-0.003,-0.001
factor2_ff5mom,4,6,-0.007,-0.008,-0.007
factor2_ff3,4,3,-0.014,-0.002,-0.008
"[y,g,v,h]",4,4,-0.030,0.014,-0.008
fund1_vix,4,3,-0.007,-0.011,-0.009
factor1,3,2,-0.018,-0.004,-0.011
factor1_vix,4,3,-0.019,-0.004,-0.012
[y],4,1,-0.009,-0.016,-0.013


In [20]:
# Relevance of config per model

spread = pt_sel.groupby('config')['avg'].agg(['mean','std','min','max'])
spread['range'] = spread['max'] - spread['min']

# sort by mean (descending)
spread = spread.sort_values('mean', ascending=False)

# style with colors on mean
styled_spread = (
    spread.style
          .format('{:.3f}')
          .background_gradient(subset=['mean'], cmap='RdYlGn', vmin=-0.1, vmax=0.1)
)
styled_spread

,mean,std,min,max,range
config,,,,,
factor2_ff3mom,-0.007,0.009,-0.014,-0.001,0.013
factor2_ff3,-0.014,0.009,-0.020,-0.008,0.012
fund2_vix,-0.016,0.003,-0.018,-0.014,0.004
fund1_vix,-0.017,0.011,-0.024,-0.009,0.015
"[g,v]",-0.017,0.002,-0.019,-0.015,0.003
fund1,-0.018,0.005,-0.021,-0.015,0.007
factor2_ff5,-0.018,0.042,-0.048,0.011,0.059
factor1,-0.018,0.010,-0.025,-0.011,0.014
"[y,g,v,h]",-0.018,0.010,-0.032,-0.008,0.024


In [22]:
# CPLL

value_sel = 'cpll (max all runs)'

res_f = res.loc[
    res['n_regimes'].isin(n_regimes_sel) & res['sector'].isin(sectors_sel),
    ['config','n_regimes','dim_latent','sector', value_sel]]

# pivot on filtered data
pt = res_f.pivot_table(
    index=['config'],
    columns='sector',
    values=value_sel,
    aggfunc='mean').sort_index().sort_index(axis=1)

# drop rows with any inf in selected sectors, then row-mean
finite_mask = np.isfinite(pt.to_numpy()).all(axis=1)
pt_sel = pt.loc[finite_mask].copy()
pt_sel['avg'] = pt_sel.mean(axis=1, skipna=True)

# sort and style
pt_sel = pt_sel.sort_values('avg', ascending=False)
styled_sel = (
    pt_sel.style
         .format('{:.3f}')
         .background_gradient(cmap='RdYlGn', vmin=-50000, vmax=50000)
         .highlight_null())

styled_sel

sector,IT,TC,avg
config,,,
fund3_vix,35147.456,35262.974,35205.215
fund3,31112.090,31225.848,31168.969
fund2_vix,26318.875,26063.383,26191.129
fund2,22282.281,22025.677,22153.979
fund1_vix,12877.424,13344.414,13110.919
factor1_vix,12787.591,13339.226,13063.409
factor1,8749.252,9300.887,9025.069
fund1,8421.191,8777.972,8599.581
factor2_ff3,-5442.438,-4850.409,-5146.424


In [24]:
# ELBO

value_sel = 'elbo_delta (max all runs)'

res_f = res.loc[
    res['n_regimes'].isin(n_regimes_sel) & res['sector'].isin(sectors_sel),
    ['config','n_regimes','dim_latent','sector', value_sel]]

# pivot on filtered data
pt = res_f.pivot_table(
    index=['config'],
    columns='sector',
    values=value_sel,
    aggfunc='mean').sort_index().sort_index(axis=1)

# drop rows with any inf in selected sectors, then row-mean
finite_mask = np.isfinite(pt.to_numpy()).all(axis=1)
pt_sel = pt.loc[finite_mask].copy()
pt_sel['avg'] = pt_sel.mean(axis=1, skipna=True)

# sort and style
pt_sel = pt_sel.sort_values('avg', ascending=False)
styled_sel = (
    pt_sel.style
         .format('{:.3f}')
         .background_gradient(cmap='RdYlGn', vmin=-50000, vmax=50000)
         .highlight_null())

styled_sel

sector,IT,TC,avg
config,,,
"[y,g,v,h]",514512.296,522010.119,518261.208
"[g,v,h]",380967.254,380465.366,380716.310
"[y,h]",355890.990,356797.290,356344.140
"[g,v]",268134.496,271139.515,269637.005
[y],139741.950,138318.463,139030.207
factor2_ff5mom,19818.043,19866.156,19842.100
factor2_ff5,16685.888,16783.100,16734.494
factor2_ff3mom,14141.224,14159.076,14150.150
factor2_ff3,10745.545,10811.192,10778.368


In [106]:
# --------------------------------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------------------------------

PATHS = {
    "data_excel": "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx",
    "ff_dir": "/Users/chrismader/Python/SLDS/Data/",
    "ff_files": {
        "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
        "ff3": "F-F_Research_Data_Factors_daily.csv",
        "mom": "F-F_Momentum_Factor_daily.csv",},
    "results_csv": "/Users/chrismader/Python/SLDS/Out/gridsearch_results.csv",
    "segments_parquet": "/Users/chrismader/Python/SLDS/Out/gridsearch_segments.parquet",
    "tmp_dir":          "/Users/chrismader/Python/SLDS/tmp_slds/",
    "segments_tmp_csv": "/Users/chrismader/Python/SLDS/tmp_slds/segments_tmp.csv",
}

CONFIG = {
    
    # Core defaults
    "n_jobs": -1,  # multi-threading
    "dt": 1.0 / 252.0,
    "n_iters": 50,
    "h_z": 3.0,  # CUSUM parameter
    
    # Batch windows
    "batch_grid": [
        {"train_window": 1260, "overlap_window": 1260-252},
    ],

    # Number of regimes
    "K_grid": [3],
    
    # Unrestricted models: 
    "unrestricted_models": [
        # {"label": "[y]",         "channels": ["y"],                "dim_latent": [1]},
        # {"label": "[y,h]",       "channels": ["y","h"],            "dim_latent": [2]},
        # {"label": "[g,v]",       "channels": ["g","v"],            "dim_latent": [2]},
        {"label": "[g,v,h]",     "channels": ["g","v","h"],        "dim_latent": [2,3]},
        # {"label": "[y,g,v,h]",   "channels": ["y","g","v","h"],    "dim_latent": [3,4]},
    ],

    # Restricted models: 
    "restricted_models": [
        # {"label": "fund1",        "channels": ["y"],                 "dim_latent": [2],    "C_type": "fund1"},
        # {"label": "fund1_vix",    "channels": ["y","h"],             "dim_latent": [3],    "C_type": "fund1_vix"},
        # {"label": "fund2",        "channels": ["y","g"],             "dim_latent": [2],    "C_type": "fund2"},
        # {"label": "fund2_vix",    "channels": ["y","g","h"],         "dim_latent": [3],    "C_type": "fund2_vix"},
        # {"label": "fund3",        "channels": ["y","v","g"],         "dim_latent": [2],    "C_type": "fund3"},
        # {"label": "fund3_vix",    "channels": ["y","v","g","h"],     "dim_latent": [3],    "C_type": "fund3_vix"},   

        # {"label": "factor1",      "channels": ["y"],                 "dim_latent": [2],    "C_type": "factor1"},
        # {"label": "factor1_vix",  "channels": ["y","h"],             "dim_latent": [3],    "C_type": "factor1_vix"},

        {"label": "factor2_ff3",   "channels": ["y","mkt","smb","hml"],                   "dim_latent": [3], "C_type": "factor2"},
        # {"label": "factor2_ff3mom","channels": ["y","mkt","smb","hml","mom"],             "dim_latent": [4], "C_type": "factor2"},
        # {"label": "factor2_ff5",   "channels": ["y","mkt","smb","hml","rmw","cma"],       "dim_latent": [5], "C_type": "factor2"},
        # {"label": "factor2_ff5mom","channels": ["y","mkt","smb","hml","rmw","cma","mom"], "dim_latent": [6], "C_type": "factor2"},
    ],

    # Model selection
    "run_unrestricted": False,
    "run_restricted": True,
    "predict_oos": True,

    # Output
    "verbose": False,
    "display": False,
}

for k, v in PATHS.items(): 
    CONFIG[k] = v
# per-security temp file templates used by IOManager
CONFIG["tmp_results_fmt"]  = "{tmp_dir}/tmp_res_{security}.csv"
CONFIG["tmp_segments_fmt"] = "{tmp_dir}/tmp_seg_{security}.csv"

In [110]:
def collect_seed_rolling_params(cfg, securities, n_runs=10, master_seed=123):
    """
    PURE WRAPPER with stability aggregation:
      • Per security → rolling batches → per-seed fits (unchanged logic).
      • NEW: For each batch, aggregate params across seeds (mean/std elementwise)
             for {Cs, ds, inv_etas, As, bs, sigmasq, Rs, r}.
      • NEW: For each batch, compute regime occupancies averaged across seeds
             for {zhat_raw, zhat_cusum} and store ARI (mean pairwise across seeds).
    The return structure now includes per-batch "params_agg" and "occ".
    """
    import numpy as np
    from gridsearch import import_data, import_factors, _intersect_indexes, _build_restrictions
    from rSLDS import fit_rSLDS, fit_rSLDS_restricted, cusum_overlay

    # pick active model (CONFIG guarantees mutual exclusivity)
    assert bool(cfg.get("run_unrestricted", False)) ^ bool(cfg.get("run_restricted", False)), \
       "[CFG] Exactly one of run_unrestricted or run_restricted must be True."
    
    if cfg["run_unrestricted"]:
        assert len(cfg["unrestricted_models"]) > 0, \
            "[CFG] run_unrestricted=True but unrestricted_models is empty."
        m = cfg["unrestricted_models"][0]; restricted = False; C_type = None
    else:
        assert len(cfg["restricted_models"]) > 0, \
            "[CFG] run_restricted=True but restricted_models is empty."
        m = cfg["restricted_models"][0];  restricted = True;  C_type = m["C_type"]
        
    channels   = list(m["channels"])
    D          = int(m["dim_latent"][0])
    K          = int(cfg["K_grid"][0])
    dt         = float(cfg["dt"])
    n_iters    = int(cfg["n_iters"])
    h_z        = float(cfg["h_z"])
    batch_conf = cfg["batch_grid"][0]
    train_window = int(batch_conf["train_window"])
    overlap      = int(batch_conf["overlap_window"])

    print(f"[CFG] restricted={restricted} C_type={C_type} channels={channels} K={K} D={D} "
          f"dt={dt} n_iters={n_iters} h_z={h_z} train_window={train_window} overlap={overlap} n_runs={n_runs}")

    # data
    px_all, eps_all, pe_all, ser_vix = import_data(cfg["data_excel"])
    ff = import_factors(cfg["ff_dir"], cfg["ff_files"])

    def _series_by_key_for(sec):
        ser_px  = px_all[sec].dropna()
        ser_eps = eps_all[sec].dropna().where(lambda s: s > 0).dropna()
        ser_pe  = (ser_px / ser_eps).where(lambda s: s > 0).dropna()
        series = {
            "y": np.log(ser_px).diff().dropna(),
            "g": np.log(ser_eps).diff().dropna(),
            "v": np.log(ser_pe).diff().dropna(),
            "h": np.log(ser_vix).diff().dropna(),
        }
        for col, key in [("MKT","mkt"),("SMB","smb"),("HML","hml"),("RMW","rmw"),("CMA","cma"),("MOM","mom")]:
            if col in ff.columns:
                series[key] = ff[col]
        return series, ser_px

    rng = np.random.RandomState(master_seed)
    out = {}

    # ---------- small helpers ----------
    def _stack(arrs, expected_ndim=None):
        A = np.stack(arrs, axis=0)  # (R, ...)
        if expected_ndim is not None:
            assert A.ndim == expected_ndim + 1, f"[ASSERT] rank mismatch: got {A.ndim}, want {expected_ndim+1}"
        return A

    def _agg_mean_std(A):
        mu = A.mean(axis=0)
        sd = A.std(axis=0, ddof=1) if A.shape[0] > 1 else np.zeros_like(mu)
        return mu, sd

    def _occ_by_seed(Z, K_):
        # Z: shape (R, T) ints in {0..K-1} or {1..K}
        if Z.size == 0:
            return np.full((Z.shape[0], K_), np.nan)
        Zp = Z.copy()
        if Zp.min() == 1:
            Zp = Zp - 1
        R, T = Zp.shape
        occ = np.zeros((R, K_), float)
        for i in range(R):
            bc = np.bincount(Zp[i], minlength=K_).astype(float)
            occ[i] = bc / max(T, 1)
        return occ

    from sklearn.metrics import adjusted_rand_score as _ARI
    def _mean_pairwise_ARI(Z):
        Z = np.asarray(Z, dtype=int)
        if Z.size == 0:
            return np.nan
        if Z.min() == 1:
            Z = Z - 1
        R, _ = Z.shape
        vals = []
        for i in range(R):
            for j in range(i+1, R):
                vals.append(_ARI(Z[i], Z[j]))
        return float(np.mean(vals)) if vals else np.nan

    # ------------------------------------

    for sec in securities:

        print(f"\n[SEC] {sec}")
        series_by_key, ser_px = _series_by_key_for(sec)
        need = [series_by_key[k].dropna() for k in channels]
        common_idx = _intersect_indexes(need)
        if common_idx is None or getattr(common_idx, "empty", False):
            print(f"[WARN] {sec}: no overlap for channels {channels} — skipping")
            continue

        Y_full  = np.concatenate([series_by_key[k].loc[common_idx].values.reshape(-1,1) for k in channels], axis=1)
        px_full = ser_px.loc[common_idx].astype(float)
        T, N = Y_full.shape
        assert N == len(channels), f"[ASSERT] N={N} must equal len(channels)={len(channels)}"

        print(f"[DATA] T={T} N={N} idx[{common_idx[0]} → {common_idx[-1]}]")

        # batch schedule
        t0s, t1s, t0 = [], [], 0
        while t0 < T:
            t1s.append(min(t0 + train_window, T))
            t0s.append(t0)
            if t1s[-1] == T: break
            t0 += train_window - overlap
        print(f"[BATCH] batches={len(t0s)}")

        rec = {
            "meta": {
                "channels": channels, "K": K, "D": D,
                "train_window": train_window, "overlap_window": overlap,
                "common_index": common_idx, "restricted": restricted, "dt": dt},
            "batches": []}

        params_spec = dict(n_regimes=K, dim_latent=D, single_subspace=True)

        for bi, (b0, b1) in enumerate(zip(t0s, t1s)):
            Yb = Y_full[b0:b1, :]
            pxb = px_full.iloc[b0:b1]
            idx_slice = pxb.index
            print(f"[B{bi}] slice t0={b0} t1={b1} len={len(idx_slice)}")

            restrictions = None
            if restricted:
                restrictions = _build_restrictions(C_type, Y_obs=Yb, base_channels=channels, D=D)
                assert "C" in restrictions and "d" in restrictions, "[ASSERT] restrictions must contain 'C' and 'd'"
                print(f"[B{bi}] restrictions keys={list(restrictions.keys())}")

            seeds = rng.randint(1, 2**31 - 1, size=n_runs).tolist()
          
            per_seed_records = []
            Z_raw_list, Z_cusum_list = [], []
            Cs_list, ds_list, inv_list = [], [], []
            As_list, bs_list, sigmasq_list, Rs_list, r_list = [], [], [], [], []

            for si, s in enumerate(seeds):
                
                if restricted:
                    xhat, zhat_raw, elbo, q, mdl = fit_rSLDS_restricted(
                        Yb, params_spec,
                        C=restrictions["C"], d=restrictions["d"],
                        n_iter_em=n_iters, seed=s,
                        b_pattern=restrictions.get("b_pattern"),
                        enforce_diag_A=True,
                        C_mask=restrictions.get("C_mask"),
                        d_mask=restrictions.get("d_mask"))
                else:
                    xhat, zhat_raw, elbo, q, mdl = fit_rSLDS(Yb, params_spec, n_iter_em=n_iters, seed=s)

                y_cus = Yb[:, 0].ravel()
                zhat_cusum = cusum_overlay(pxb, y_cus, xhat, mdl, h_z)

                Z_raw_list.append(np.asarray(zhat_raw).ravel())
                if zhat_cusum is None:
                    Z_cusum_list.append(np.asarray(zhat_raw).ravel())
                else:
                    Z_cusum_list.append(np.asarray(zhat_cusum).ravel())

                # params (copy to detach)
                Cs      = np.array(mdl.emissions.Cs,       copy=True)
                ds      = np.array(mdl.emissions.ds,       copy=True)
                inv_eta = np.array(mdl.emissions.inv_etas, copy=True)
                As      = np.array(mdl.dynamics.As,        copy=True)
                bs      = np.array(mdl.dynamics.bs,        copy=True)
                sigmasq = np.array(mdl.dynamics.sigmasq,   copy=True)
                Rs      = np.array(mdl.transitions.Rs,     copy=True)
                r       = np.array(mdl.transitions.r,      copy=True)

                # sanity on leading K (allow shared emissions with leading 1)
                assert As.shape[0] == K and bs.shape[0] == K and sigmasq.shape[0] == K \
                    and Rs.shape[0] == K and r.shape[0] == K, "[ASSERT] K mismatch in dynamics/transitions"
                assert Cs.ndim == 3 and ds.ndim == 2 and inv_eta.ndim == 2, "[ASSERT] emissions rank mismatch"
                assert Cs.shape[0] in (1, K) and ds.shape[0] in (1, K) and inv_eta.shape[0] in (1, K), \
                    "[ASSERT] emissions leading axis must be 1 or K"

                Cs_list.append(Cs)
                ds_list.append(ds)
                inv_list.append(inv_eta)
                As_list.append(As)
                bs_list.append(bs)
                sigmasq_list.append(sigmasq)
                Rs_list.append(Rs)
                r_list.append(r)

                per_seed_records.append({
                    "seed": int(s),
                    "params": {"Cs": Cs, "ds": ds, "inv_etas": inv_eta, "As": As, "bs": bs,
                               "sigmasq": sigmasq, "Rs": Rs, "r": r},
                    "outputs": {"zhat_raw": zhat_raw, "zhat_cusum": zhat_cusum,
                                "xhat": xhat, "elbo": elbo}
                })

            # ---------- aggregate across seeds (elementwise mean/std) ----------
            def _agg_block(block_list):
                A = _stack(block_list)  # (R, ...)
                mu, sd = _agg_mean_std(A)
                return {"mean": mu, "std": sd}

            params_agg = {
                "Cs": _agg_block(Cs_list),
                "ds": _agg_block(ds_list),
                "inv_etas": _agg_block(inv_list),
                "As": _agg_block(As_list),
                "bs": _agg_block(bs_list),
                "sigmasq": _agg_block(sigmasq_list),
                "Rs": _agg_block(Rs_list),
                "r": _agg_block(r_list),
            }

            # ---------- regime occupancy (avg across seeds) + ARI ----------
            Zr = np.stack(Z_raw_list, axis=0)   if len(Z_raw_list)  else np.zeros((0, 0), int)
            Zc = np.stack(Z_cusum_list, axis=0) if len(Z_cusum_list) else np.zeros((0, 0), int)

            occ_raw_seed = _occ_by_seed(Zr, K)  # (R,K)
            occ_cus_seed = _occ_by_seed(Zc, K)  # (R,K)

            occ = {
                "raw": {
                    "by_seed": occ_raw_seed,                      # (R,K), fractions
                    "mean":    np.nanmean(occ_raw_seed, axis=0),  # (K,)
                    "std":     np.nanstd(occ_raw_seed, axis=0, ddof=1) if occ_raw_seed.shape[0] > 1
                               else np.zeros(K)
                },
                "cusum": {
                    "by_seed": occ_cus_seed,
                    "mean":    np.nanmean(occ_cus_seed, axis=0),
                    "std":     np.nanstd(occ_cus_seed, axis=0, ddof=1) if occ_cus_seed.shape[0] > 1
                               else np.zeros(K)
                }
            }

            metrics = {
                "ARI_raw":  _mean_pairwise_ARI(Zr),
                "ARI_cusum": _mean_pairwise_ARI(Zc),
            }

            rec["batches"].append({
                "t0": int(b0), "t1": int(b1),
                "idx_slice": idx_slice,
                "seeds": seeds,
                "records": per_seed_records,
                "Z_raw":  Zr,
                "Z_cusum": Zc,
                "params_agg": params_agg,
                "occ": occ,
                "metrics": metrics,
            })
        
        out[sec] = rec

    return out


In [114]:
def print_seed_rolling_stability(rolled, security, batch_indices=None):
    """
    Pretty printer for stability:
      • Emissions & latent blocks: entries as mean (std) across seeds.
      • Regime occupancies per batch: mean% (std%).
      • ARI per batch: raw & cusum.
    Inputs
      rolled: dict returned by collect_seed_rolling_params
      security: key inside rolled
      batch_indices: optional list of batch indices to print (default: all)
    """
    import numpy as np

    rec = rolled.get(security)
    assert rec is not None, f"[print] security '{security}' not found"
    K = rec["meta"]["K"]; D = rec["meta"]["D"]; dt = rec["meta"]["dt"]; restricted = rec["meta"]["restricted"]
    B = len(rec["batches"])
    if batch_indices is None:
        batch_indices = list(range(B))

    def _fmt(m, s, fp=4):
        return f"{m: .{fp}f} ({s: .{fp}f})"

    def _pmat(mu, sd, label, semantic_key):
        shape_str = {
            "Cs": "(K_or_1, N, D)",
            "ds": "(K_or_1, N)",
            "inv_etas": "(K_or_1, N)",
            "As": "(K, D, D)",
            "bs": "(K, D)",
            "sigmasq": "(K, D)",
            "Rs": "(K, D)",
            "r": "(K,)",
        }[semantic_key]
        print(f"{label}  {shape_str} = {mu.shape}")
        if mu.ndim == 3:
            K0, R0, C0 = mu.shape
            for r in range(R0):
                line = ""
                for k in range(K0):
                    row_entries = []
                    for c in range(C0):
                        row_entries.append(_fmt(mu[k, r, c], sd[k, r, c]))
                    row_str = " ".join(row_entries)
                    line += f"[{row_str}]    "
                print(line)
        elif mu.ndim == 2:
            K0, C0 = mu.shape
            line = ""
            for k in range(K0):
                row_entries = [_fmt(mu[k, c], sd[k, c]) for c in range(C0)]
                line += f"[{' '.join(row_entries)}]" + "    "
            print(line)
        elif mu.ndim == 1:
            row_entries = [_fmt(mu[k], sd[k]) for k in range(mu.shape[0])]
            print(f"[{' '.join(row_entries)}]")
        print()

    for bi in batch_indices:
        b = rec["batches"][bi]
        t0, t1 = b["t0"], b["t1"]
        print("\n========== STABILITY (security={}, batch={}  t0={}  t1={}) ==========\n"
              .format(security, bi, t0, t1))

        P = b["params_agg"]
        _pmat(P["Cs"]["mean"],      P["Cs"]["std"],      "Cs",       "Cs")
        _pmat(P["ds"]["mean"],      P["ds"]["std"],      "ds",       "ds")
        _pmat(P["inv_etas"]["mean"],P["inv_etas"]["std"],"inv_etas", "inv_etas")
        _pmat(P["As"]["mean"],      P["As"]["std"],      "As",       "As")
        _pmat(P["bs"]["mean"],      P["bs"]["std"],      "bs",       "bs")
        _pmat(P["sigmasq"]["mean"], P["sigmasq"]["std"], "sigmasq",  "sigmasq")
        _pmat(P["Rs"]["mean"],      P["Rs"]["std"],      "Rs",       "Rs")
        _pmat(P["r"]["mean"],       P["r"]["std"],       "r",        "r")

        # interpretable AR(1) params (restricted models)
        if restricted:
            print("Interpretable AR(1) parameters (mean±std over seeds)")
            mu_As, sd_As   = P["As"]["mean"], P["As"]["std"]         # (K,D,D)
            mu_bs, sd_bs   = P["bs"]["mean"], P["bs"]["std"]         # (K,D)
            mu_sig, sd_sig = P["sigmasq"]["mean"], P["sigmasq"]["std"] # (K,D)
            eps = 1e-8
            for k in range(mu_As.shape[0]):
                print(f"Regime {k}:")
                for d in range(min(D, mu_As.shape[2])):
                    rho_m, rho_s = mu_As[k, d, d], sd_As[k, d, d]
                    b_m, b_s     = mu_bs[k, d],    sd_bs[k, d]
                    # μ = b / (1-ρ), Var_stat = σ²/(1-ρ²) for |ρ|<1
                    mu_stat_m = (b_m / (1.0 - rho_m)) if abs(1.0 - rho_m) > eps else np.nan
                    # delta-method-ish SD approximation (optional: coarse; keep concise)
                    mu_stat_s = np.nan
                    s2_m, s2_s = mu_sig[k, d], sd_sig[k, d]
                    var_stat_m = (s2_m / (1.0 - rho_m**2)) if abs(rho_m) < 1 else np.nan
                    var_stat_s = np.nan
                    # half-life in steps: log(2)/|log ρ|
                    if rho_m > 0 and abs(rho_m) < 1:
                        hl_steps_m = float(np.log(2.0) / abs(np.log(rho_m)))
                    else:
                        hl_steps_m = float("inf")
                    hl_years_m = (hl_steps_m * rec["meta"]["dt"]) if np.isfinite(hl_steps_m) else np.nan
                    print(f"  latent{d}: rho={_fmt(rho_m, rho_s)}  b={_fmt(b_m, b_s)}  "
                          f"mu_stat≈{mu_stat_m: .6f}  var_stat≈{var_stat_m: .6e}  "
                          f"half_life_steps≈{hl_steps_m: .2f}  half_life_years≈{hl_years_m: .4f}")
                print()

        # occupancy + ARI
        occ = b["occ"]
        raw_mean = (occ["raw"]["mean"] * 100.0).astype(float)
        raw_std  = (occ["raw"]["std"]  * 100.0).astype(float)
        cus_mean = (occ["cusum"]["mean"] * 100.0).astype(float)
        cus_std  = (occ["cusum"]["std"]  * 100.0).astype(float)

        def _fmt_occ(mean_vec, std_vec):
            return ", ".join([f"{mean_vec[j]:.1f}% ({std_vec[j]:.1f}%)" for j in range(mean_vec.size)])

        print("Occupancy per regime (mean% (std%) across seeds):")
        print(f"  RAW:   [{_fmt_occ(raw_mean,  raw_std)}]")
        print(f"  CUSUM: [{_fmt_occ(cus_mean, cus_std)}]")

        ARI_raw   = b["metrics"]["ARI_raw"]
        ARI_cusum = b["metrics"]["ARI_cusum"]
        print(f"ARI (mean pairwise across seeds):  RAW={ARI_raw:.3f}   CUSUM={ARI_cusum:.3f}")


In [ ]:
securities = ["WMT"]
rolled = collect_seed_rolling_params(CONFIG, securities, n_runs=10, master_seed=123)
print_seed_rolling_stability(rolled, "WMT")

[CFG] restricted=True C_type=factor2 channels=['y', 'mkt', 'smb', 'hml'] K=3 D=3 dt=0.003968253968253968 n_iters=50 h_z=3.0 train_window=1260 overlap=63 n_runs=10

[SEC] WMT
[DATA] T=3392 N=4 idx[2012-01-03 00:00:00 → 2025-06-30 00:00:00]
[BATCH] batches=3
[B0] slice t0=0 t1=1260 len=1260
[B0] restrictions keys=['C', 'd', 'b_pattern']
